# Ransomware Static Analysis

# Preparation

In [1]:
! pip install bottleneck numexpr pandas pefile

In [2]:
import glob
import numpy as np
import os
import pandas as pd
import pefile

In [3]:
ransomware_path = "C:/sample/ransomware/"
system32_path = "C:/sample/good/System32/"

In [4]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)
# pd.set_option('display.width', 1000)
# pd.set_option('max_colwidth', 400)

In [5]:
pd.reset_option("display.max_rows")
pd.reset_option("display.max_columns")
pd.reset_option("display.width")
pd.reset_option("display.max_colwidth")

# Define Functions

In [5]:
# Define function to create dataframes

def CreateDataFrame(path, attribute):
    df = pd.DataFrame()
    filesPath = glob.glob(path + "*")
    nameFiles = os.listdir(path)
    index = 0
    count = 0
    for filePath in filesPath:
        # print(file)
        pe = pefile.PE(filePath)
        if attribute == "FILE_HEADER":
            df.loc[index,'Name'] = nameFiles[index]
            for k, v in pe.dump_dict()['FILE_HEADER'].items():
                if isinstance(v,dict):
                    # print("{}: {}".format(k, v["Value"]))
                    df.loc[index, k] = v["Value"]
            # index = 0
            for k, v in pe.dump_dict()['OPTIONAL_HEADER'].items():
                if isinstance(v,dict):
                    # print("{}\n{}: {}".format(filePath, k, v["Value"]))
                    df.loc[index, k] = v["Value"]
            index += 1
        elif attribute == "DIRECTORY_ENTRY_IMPORT":
            try:
                for entry in pe.DIRECTORY_ENTRY_IMPORT:
                    for function in entry.imports:
                        if function.name != None:
                            # print("File:", filePath, "Dll", entry.dll, "API name:", api_call.name)
                            df.loc[count, 'Name'] = nameFiles[index]
                            df.loc[count, 'Dll'] = entry.dll
                            df.loc[count, 'Function'] = function.name
                            count += 1
                    # count += 1
                index += 1
            except AttributeError:
                print("File", filePath, "does not have imports.")
    return df

# Manipulating ransomware dataframe

In [6]:
# Create ransomware dataframe
ransomware_df = CreateDataFrame(ransomware_path, "FILE_HEADER")
#ransomware_df.to_csv(ransomware_path + "../ransomware_df.csv", index=False)

In [7]:
# Read ransomware CSV file
ransomware_df = pd.read_csv("ransomware_df.csv")

In [8]:
print(ransomware_df.shape)
ransomware_df.head()

(24, 38)


Name  Machine  NumberOfSections  \
0         131.sample    332.0               5.0   
1     Avaddon.sample    332.0               5.0   
2      cerber.sample    332.0               4.0   
3  cryptowall.sample    332.0               4.0   
4      Crysis.sample    332.0               3.0   

                               TimeDateStamp  PointerToSymbolTable  \
0  0x571C6108 [Sun Apr 24 06:00:40 2016 UTC]          0.000000e+00   
1  0x5D652AF9 [Tue Aug 27 13:07:05 2019 UTC]          0.000000e+00   
2  0x5925F1A2 [Wed May 24 20:48:34 2017 UTC]          0.000000e+00   
3  0x54B4D689 [Tue Jan 13 08:25:45 2015 UTC]          0.000000e+00   
4  0x58B8AF72 [Thu Mar  2 23:49:06 2017 UTC]          1.919897e+09   

   NumberOfSymbols  SizeOfOptionalHeader  Characteristics  Magic  \
0     0.000000e+00                 224.0            258.0  267.0   
1     0.000000e+00                 224.0            259.0  267.0   
2     0.000000e+00                 224.0            259.0  267.0   
3     0.000000e+00                 224.0            259.0  267.0   
4     1.564824e+09                 224.0            259.0  267.0   

   MajorLinkerVersion  MinorLinkerVersion  SizeOfCode  SizeOfInitializedData  \
0                11.0                 0.0    122880.0              2299392.0   
1                12.0                 0.0     69632.0             49323520.0   
2                 9.0                 0.0    323072.0               294912.0   
3                 9.0                 0.0     34816.0               210432.0   
4                10.0                 0.0     40448.0                54272.0   

   SizeOfUninitializedData  AddressOfEntryPoint  BaseOfCode  BaseOfData  \
0                      0.0              33303.0      4096.0    126976.0   
1                      0.0              22283.0      4096.0     73728.0   
2                      0.0             324832.0      4096.0    327680.0   
3                      0.0              13447.0      4096.0     40960.0   
4                      0.0              43472.0      4096.0     45056.0   

   ImageBase  SectionAlignment  FileAlignment  MajorOperatingSystemVersion  \
0  4194304.0            4096.0          512.0                          5.0   
1  4194304.0            4096.0          512.0                          5.0   
2  4194304.0            4096.0          512.0                          5.0   
3  4194304.0            4096.0          512.0                          5.0   
4  4194304.0            4096.0          512.0                          5.0   

   MinorOperatingSystemVersion  MajorImageVersion  MinorImageVersion  \
0                          1.0                0.0                0.0   
1                          1.0                0.0                0.0   
2                          0.0                0.0                0.0   
3                          0.0                0.0                0.0   
4                          1.0                0.0                0.0   

   MajorSubsystemVersion  MinorSubsystemVersion  Reserved1  SizeOfImage  \
0                    5.0                    1.0        0.0    2433024.0   
1                    5.0                    1.0        0.0   49397760.0   
2                    5.0                    0.0        0.0     630784.0   
3                    5.0                    0.0        0.0   54423552.0   
4                    5.0                    1.0        0.0     101077.0   

   SizeOfHeaders  CheckSum  Subsystem  DllCharacteristics  SizeOfStackReserve  \
0         1024.0       0.0        3.0             33088.0           1048576.0   
1         1024.0  756569.0        2.0             32768.0           1048576.0   
2         1024.0       0.0        2.0             32768.0           1048576.0   
3         1024.0       0.0        2.0             32768.0           1048576.0   
4          512.0       0.0        2.0             34048.0           1048576.0   

   SizeOfStackCommit  SizeOfHeapReserve  SizeOfHeapCommit  LoaderFlags  \
0             4096.0          104

In [45]:
# Create ransomware functions dataframe
ransomware_functions = CreateDataFrame(ransomware_path, "DIRECTORY_ENTRY_IMPORT")
# ransomware_functions.to_csv(ransomware_path + "../ransomware_functions.csv", index=False)

File C:/sample/ransomware\Thanos.sample does not have imports.


In [9]:
# Read ransomware fuctions CSV file
ransomware_functions = pd.read_csv("ransomware_functions.csv")

In [10]:
ransomware_functions

Name              Dll             Function
0          131.sample   b'SHLWAPI.dll'   b'PathFileExistsW'
1          131.sample   b'SHLWAPI.dll'   b'PathFileExistsA'
2          131.sample  b'KERNEL32.dll'             b'Sleep'
3          131.sample  b'KERNEL32.dll'    b'SizeofResource'
4          131.sample  b'KERNEL32.dll'  b'GetConsoleWindow'
...               ...              ...                  ...
2071  Wannacry.sample    b'MSVCRT.dll'              b'free'
2072  Wannacry.sample    b'MSVCRT.dll'         b'_initterm'
2073  Wannacry.sample    b'MSVCRT.dll'            b'malloc'
2074  Wannacry.sample    b'MSVCRT.dll'      b'_adjust_fdiv'
2075  Wannacry.sample    b'MSVCRT.dll'           b'sprintf'

[2076 rows x 3 columns]

In [11]:
# Create System32 dataframe
system32_df = CreateDataFrame(system32_path, "FILE_HEADER")
# system32_df.to_csv(system32_path + "../system32_df.csv", index=False)

In [11]:
# Read System32 CSV file
system32_df = pd.read_csv("system32_df.csv")

In [12]:
system32_df

Name  Machine  NumberOfSections  \
0           aadcloudap.dll  34404.0               7.0   
1                aadtb.dll  34404.0               7.0   
2     AboveLockAppHost.dll  34404.0               7.0   
3     accessibilitycpl.dll  34404.0               7.0   
4      accountaccessor.dll  34404.0               7.0   
...                    ...      ...               ...   
3485           xwtpdui.dll  34404.0               7.0   
3486           xwtpw32.dll  34404.0               7.0   
3487      zipcontainer.dll  34404.0               6.0   
3488           zipfldr.dll  34404.0               7.0   
3489       ztrace_maps.dll  34404.0               6.0   

                                  TimeDateStamp  PointerToSymbolTable  \
0     0x5CCD15A1 [Sat May  4 04:31:29 2019 UTC]                   0.0   
1     0x60BAFDF1 [Sat Jun  5 04:30:41 2021 UTC]                   0.0   
2     0x5FF78DB7 [Thu Jan  7 22:39:51 2021 UTC]                   0.0   
3     0x57899883 [Sat Jul 16 02:14:27 2016 UTC]                   0.0   
4     0x5FF78E70 [Thu Jan  7 22:42:56 2021 UTC]                   0.0   
...                                         ...                   ...   
3485  0x5FF78DCC [Thu Jan  7 22:40:12 2021 UTC]                   0.0   
3486  0x5FF78E15 [Thu Jan  7 22:41:25 2021 UTC]                   0.0   
3487  0x57899BDE [Sat Jul 16 02:28:46 2016 UTC]                   0.0   
3488  0x5FF78DA9 [Thu Jan  7 22:39:37 2021 UTC]                   0.0   
3489  0x57899BE0 [Sat Jul 16 02:28:48 2016 UTC]                   0.0   

      NumberOfSymbols  SizeOfOptionalHeader  Characteristics  Magic  \
0                 0.0                 240.0           8226.0  523.0   
1                 0.0                 240.0           8226.0  523.0   
2                 0.0                 240.0           8226.0  523.0   
3                 0.0                 240.0           8226.0  523.0   
4                 0.0                 240.0           8226.0  523.0   
...               ...                   ...              ...    ...   
3485              0.0                 240.0           8226.0  523.0   
3486              0.0                 240.0           8226.0  523.0   
3487              0.0                 240.0           8226.0  523.0   
3488              0.0                 240.0           8226.0  523.0   
3489              0.0                 240.0           8226.0  523.0   

      MajorLinkerVersion  MinorLinkerVersion  SizeOfCode  \
0                   14.0                 0.0    273920.0   
1                   14.0                 0.0    620032.0   
2                   14.0                 0.0    180224.0   
3                   14.0                 0.0    145920.0   
4                   14.0                 0.0    226304.0   
...                  ...                 ...         ...   
3485                14.0                 0.0    151040.0   
3486                14.0                 0.0     83968.0   
3487                14.0                 0.0     54784.0   
3488                14.0                 0.0    212480.0   
3489                14.0                 0.0     12288.0   

      SizeOfInitializedData  SizeOfUninitializedData  AddressOfEntryPoint  \
0                  153600.0                      0.0              19472.0   
1                  505344.0                      0.0             527744.0   
2                  107520.0                      0.0              33072.0   
3                 3691520.0                      0.0               5680.0   
4                   97792.0                      0.0             223920.0   
...                     ...                      ...                  ...   
3485               108544.0                      0.0             144048.0   
3486                62464.0                      0.0              80800.0   
3487                25600.0                      0.0              54768.0   
3488               178176.0                      0.0             212256.0   
3489                19456.0            

In [12]:
# Create system32 functions dataframe
system32_functions = CreateDataFrame(system32_path, "DIRECTORY_ENTRY_IMPORT")
# system32_functions.to_csv(system32_path + "../system32_functions.csv", index=False)

File C:/sample/good/System32\ACCTRES.dll does not have imports.
File C:/sample/good/System32\adtschema.dll does not have imports.
File C:/sample/good/System32\advapi32res.dll does not have imports.
File C:/sample/good/System32\aeevts.dll does not have imports.
File C:/sample/good/System32\APHostRes.dll does not have imports.
File C:/sample/good/System32\apisetschema.dll does not have imports.
File C:/sample/good/System32\AppvClientEventLog.dll does not have imports.
File C:/sample/good/System32\appvetwclientres.dll does not have imports.
File C:/sample/good/System32\appvetwsharedperformance.dll does not have imports.
File C:/sample/good/System32\appvetwstreamingux.dll does not have imports.
File C:/sample/good/System32\AppVStreamingUX.exe does not have imports.
File C:/sample/good/System32\asferror.dll does not have imports.
File C:/sample/good/System32\auditpolmsg.dll does not have imports.
File C:/sample/good/System32\BlbEvents.dll does not have imports.
File C:/sample/good/System32\

FileNotFoundError: [WinError 2] O sistema não pode encontrar o arquivo especificado: 'C:/sample/good/System32\\system32_df.csv'

In [13]:
# Read System32 functions CSV file
system32_functions = pd.read_csv("system32_functions.csv")

In [14]:
system32_functions

Name                                        Dll  \
0            aadcloudap.dll                              b'msvcrt.dll'   
1            aadcloudap.dll                              b'msvcrt.dll'   
2            aadcloudap.dll                              b'msvcrt.dll'   
3            aadcloudap.dll                              b'msvcrt.dll'   
4            aadcloudap.dll                              b'msvcrt.dll'   
...                     ...                                        ...   
460142  wincredprovider.dll      b'api-ms-win-core-sysinfo-l1-2-1.dll'   
460143  wincredprovider.dll  b'api-ms-win-core-interlocked-l1-2-0.dll'   
460144  wincredprovider.dll   b'api-ms-win-core-rtlsupport-l1-2-0.dll'   
460145  wincredprovider.dll   b'api-ms-win-core-rtlsupport-l1-2-0.dll'   
460146  wincredprovider.dll   b'api-ms-win-core-rtlsupport-l1-2-0.dll'   

                          Function  
0                       b'_onexit'  
1                   b'__dllonexit'  
2                       b'_unlock'  
3         b'??1type_info@@UEAA@XZ'  
4                        b'_errno'  
...                            ...  
460142  b'GetSystemTimeAsFileTime'  
460143      b'InitializeSListHead'  
460144        b'RtlCaptureContext'  
460145   b'RtlLookupFunctionEntry'  
460146         b'RtlVirtualUnwind'  

[460147 rows x 3 columns]

# Merge Dataframes

## Merge Ransomware and System32 Dataframes

In [28]:
# Merge ransomware_df.csv and system32_df.csv and output in merged_df.csv
ransomware_df['label'] = 1
system32_df['label'] = 0
frames = [ransomware_df, system32_df]
merged_df = pd.concat(frames)

# To write the dataframe in disk, uncomment the line bellow
# merged_df.to_csv("merged_df.csv", index=False)

In [13]:
# Read merged ransomware and System32 Dataframes CSV file
merged_df = pd.read_csv("merged_df.csv")
merged_df

Name  Machine  NumberOfSections  \
0            131.sample    332.0               5.0   
1        Avaddon.sample    332.0               5.0   
2         cerber.sample    332.0               4.0   
3     cryptowall.sample    332.0               4.0   
4         Crysis.sample    332.0               3.0   
...                 ...      ...               ...   
3509        xwtpdui.dll  34404.0               7.0   
3510        xwtpw32.dll  34404.0               7.0   
3511   zipcontainer.dll  34404.0               6.0   
3512        zipfldr.dll  34404.0               7.0   
3513    ztrace_maps.dll  34404.0               6.0   

                                  TimeDateStamp  PointerToSymbolTable  \
0     0x571C6108 [Sun Apr 24 06:00:40 2016 UTC]          0.000000e+00   
1     0x5D652AF9 [Tue Aug 27 13:07:05 2019 UTC]          0.000000e+00   
2     0x5925F1A2 [Wed May 24 20:48:34 2017 UTC]          0.000000e+00   
3     0x54B4D689 [Tue Jan 13 08:25:45 2015 UTC]          0.000000e+00   
4     0x58B8AF72 [Thu Mar  2 23:49:06 2017 UTC]          1.919897e+09   
...                                         ...                   ...   
3509  0x5FF78DCC [Thu Jan  7 22:40:12 2021 UTC]          0.000000e+00   
3510  0x5FF78E15 [Thu Jan  7 22:41:25 2021 UTC]          0.000000e+00   
3511  0x57899BDE [Sat Jul 16 02:28:46 2016 UTC]          0.000000e+00   
3512  0x5FF78DA9 [Thu Jan  7 22:39:37 2021 UTC]          0.000000e+00   
3513  0x57899BE0 [Sat Jul 16 02:28:48 2016 UTC]          0.000000e+00   

      NumberOfSymbols  SizeOfOptionalHeader  Characteristics  Magic  \
0        0.000000e+00                 224.0            258.0  267.0   
1        0.000000e+00                 224.0            259.0  267.0   
2        0.000000e+00                 224.0            259.0  267.0   
3        0.000000e+00                 224.0            259.0  267.0   
4        1.564824e+09                 224.0            259.0  267.0   
...               ...                   ...              ...    ...   
3509     0.000000e+00                 240.0           8226.0  523.0   
3510     0.000000e+00                 240.0           8226.0  523.0   
3511     0.000000e+00                 240.0           8226.0  523.0   
3512     0.000000e+00                 240.0           8226.0  523.0   
3513     0.000000e+00                 240.0           8226.0  523.0   

      MajorLinkerVersion  MinorLinkerVersion  SizeOfCode  \
0                   11.0                 0.0    122880.0   
1                   12.0                 0.0     69632.0   
2                    9.0                 0.0    323072.0   
3                    9.0                 0.0     34816.0   
4                   10.0                 0.0     40448.0   
...                  ...                 ...         ...   
3509                14.0                 0.0    151040.0   
3510                14.0                 0.0     83968.0   
3511                14.0                 0.0     54784.0   
3512                14.0                 0.0    212480.0   
3513                14.0                 0.0     12288.0   

      SizeOfInitializedData  SizeOfUninitializedData  AddressOfEntryPoint  \
0                 2299392.0                      0.0              33303.0   
1                49323520.0                      0.0              22283.0   
2                  294912.0                      0.0             324832.0   
3                  210432.0                      0.0              13447.0   
4                   54272.0                      0.0              43472.0   
...                     ...                      ...                  ...   
3509               108544.0                      0.0             144048.0   
3510                62464.0                      0.0              80800.0   
3511                25600.0                      0.0              54768.0   
3512               178176.0                      0.0             212256.0   
3513                19456.0                      0.0              11872.

## Merge Ransomware and System32 Functions

In [20]:
ransomware_functions = pd.read_csv("ransomware_functions.csv")
# ransomware_functions['ransomware'] = 1
# ransomware_functions['good'] = 0
ransomware_functions['label'] = 1


system32_functions = pd.read_csv("system32_functions.csv")
# system32_functions['ransomware'] = 0
# system32_functions['good'] = 1
system32_functions['label'] = 0

frames = [ransomware_functions, system32_functions]
merged_functions = pd.concat(frames)

# To write the dataframe in disk, uncomment the line bellow
# merged_functions.to_csv("merged_functions.csv", index=False)

In [14]:
merged_functions = pd.read_csv("merged_functions.csv")
merged_functions

Dll                    Function  \
0                                  b'SHLWAPI.dll'          b'PathFileExistsW'   
1                                  b'SHLWAPI.dll'          b'PathFileExistsA'   
2                                 b'KERNEL32.dll'                    b'Sleep'   
3                                 b'KERNEL32.dll'           b'SizeofResource'   
4                                 b'KERNEL32.dll'         b'GetConsoleWindow'   
...                                           ...                         ...   
462218      b'api-ms-win-core-sysinfo-l1-2-1.dll'  b'GetSystemTimeAsFileTime'   
462219  b'api-ms-win-core-interlocked-l1-2-0.dll'      b'InitializeSListHead'   
462220   b'api-ms-win-core-rtlsupport-l1-2-0.dll'        b'RtlCaptureContext'   
462221   b'api-ms-win-core-rtlsupport-l1-2-0.dll'   b'RtlLookupFunctionEntry'   
462222   b'api-ms-win-core-rtlsupport-l1-2-0.dll'         b'RtlVirtualUnwind'   

        ransomware  good  
0                1     0  
1                1     0  
2                1     0  
3                1     0  
4                1     0  
...            ...   ...  
462218           0     1  
462219           0     1  
462220           0     1  
462221           0     1  
462222           0     1  

[462223 rows x 4 columns]

In [5]:
# Print all PE header information:
pe = pefile.PE(ransomware_path + "Crysis.sample")
print(pe.dump_info())

----------Parsing Warnings----------

Imported symbols contain entries typical of packed executables.

----------DOS_HEADER----------

[IMAGE_DOS_HEADER]
0x0        0x0   e_magic:                       0x5A4D    
0x2        0x2   e_cblp:                        0x40      
0x4        0x4   e_cp:                          0x1       
0x6        0x6   e_crlc:                        0x0       
0x8        0x8   e_cparhdr:                     0x2       
0xA        0xA   e_minalloc:                    0x4       
0xC        0xC   e_maxalloc:                    0xFFFF    
0xE        0xE   e_ss:                          0x2       
0x10       0x10  e_sp:                          0x40      
0x12       0x12  e_csum:                        0x0       
0x14       0x14  e_ip:                          0xE       
0x16       0x16  e_cs:                          0x0       
0x18       0x18  e_lfarlc:                      0x1C      
0x1A       0x1A  e_ovno:                        0x0       
0x1C       0x1C  e_r